In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
cd drive/My Drive/DL_endsem_improvedGANs

In [0]:
ls

In [0]:
cd Colab Notebooks

In [0]:
ls

In [0]:
! pip install tensorboardx

In [0]:
from __future__ import print_function 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from functional import *
from torch.utils.data import DataLoader,TensorDataset
import sys
import argparse
# from Datasets import *
import pdb
import tensorboardX
import os
from itertools import cycle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from scipy.spatial.distance import cdist
from sklearn.metrics import average_precision_score

In [0]:
import torch
from torch.utils.data import TensorDataset
from torchvision import datasets, transforms
import numpy as np

def MnistLabel(class_num, ntriplets):
    raw_dataset = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]))
    class_tot = [0] * 10
    data = []
    labels = []
    positive_tot = 0
    tot = 0
    
    for i in range(raw_dataset.__len__()):
        datum, label = raw_dataset.__getitem__(i)
        if class_tot[label] < class_num:
            data.append(datum.numpy())
            labels.append(label)
            class_tot[label] += 1
            tot += 1
            if tot >= 10 * class_num:
                break
    labels = np.array(labels)
    triplet_data = []
    triplet_label = []
    triplets = []
    for i in range(ntriplets):
      
      class_idx = np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 1)[0]
      

      a = np.random.choice(np.where(labels==class_idx)[0], 1)[0]
      b = np.random.choice(np.where(labels==class_idx)[0], 1)[0]
      while (a-b) == 0:
          b = np.random.choice(np.where(labels==class_idx)[0], 1)[0]
      c = np.random.choice(np.where(labels!=class_idx)[0], 1)[0]
      triplets.append([a, b, c])
    # print(labels[triplets[0][0]], labels[triplets[0][1]], labels[triplets[0][2]])
    # print(triplets[0][0])
    for i in range(0, ntriplets, 100):
      k = i
      for j in range(k, k + 100):
          triplet_data.append(data[triplets[j][0]])
          triplet_label.append(labels[triplets[j][0]])
      for j in range(k, k + 100):
          triplet_data.append(data[triplets[j][1]])
          triplet_label.append(labels[triplets[j][1]])
      for j in range(k, k + 100):
          triplet_data.append(data[triplets[j][2]])
          triplet_label.append(labels[triplets[j][2]])
    # print(len(triplet_data), len(triplet_label))
    # print(triplet_label[301], triplet_label[1])
    # print(triplet_label[401], triplet_label[101])
    # print(triplet_label[501], triplet_label[201])
    triplet_data = np.array(triplet_data)
    # print(triplet_data.dtype)
    triplet_label = np.array(triplet_label)
    return TensorDataset(torch.FloatTensor(triplet_data), torch.LongTensor(triplet_label))


def MnistUnlabel():
    raw_dataset = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]))
    return raw_dataset
def MnistTest():
    return datasets.MNIST('../data', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]))

In [0]:
# architecture taken from @https://github.com/Sleepychord/ImprovedGAN-pytorch
class Discriminator(nn.Module):
    def __init__(self, input_dim = 28 ** 2, output_dim = 16):
        super(Discriminator, self).__init__()
        self.input_dim = input_dim
        self.layers = torch.nn.ModuleList([
            LinearWeightNorm(input_dim, 1000),
            LinearWeightNorm(1000, 500),
            LinearWeightNorm(500, 250),
            LinearWeightNorm(250, 250),
            LinearWeightNorm(250, 250)]
        )
        self.final = LinearWeightNorm(250, output_dim, weight_scale=1)


    def forward(self, x, feature = False):
        x = x.view(-1, self.input_dim).cuda()
        noise = torch.randn(x.size()) * 0.3 if self.training else torch.Tensor([0])
        noise = noise.cuda()
        x = x + Variable(noise)
        for i in range(len(self.layers)):
            m = self.layers[i]
            x_f = F.relu(m(x))
            noise = torch.randn(x_f.size()) * 0.5 if self.training else torch.Tensor([0])
            noise = noise.cuda()
            x = (x_f + Variable(noise))
        if feature:
            return x_f, self.final(x)
        return self.final(x)


class Generator(nn.Module):
    def __init__(self, z_dim, output_dim = 28 ** 2):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        self.fc1 = nn.Linear(z_dim, 500, bias = False)
        self.bn1 = nn.BatchNorm1d(500, affine = False, eps=1e-6, momentum = 0.5)
        self.fc2 = nn.Linear(500, 500, bias = False)
        self.bn2 = nn.BatchNorm1d(500, affine = False, eps=1e-6, momentum = 0.5)
        self.fc3 = LinearWeightNorm(500, output_dim, weight_scale = 1)
        self.bn1_b = Parameter(torch.zeros(500))
        self.bn2_b = Parameter(torch.zeros(500))
        nn.init.xavier_uniform(self.fc1.weight)
        nn.init.xavier_uniform(self.fc2.weight)

    def forward(self, batch_size):
        x = Variable(torch.rand(batch_size, self.z_dim), requires_grad = False, volatile = not self.training)
        
        x = x.cuda()
        x = F.softplus(self.bn1(self.fc1(x)) + self.bn1_b)
        x = F.softplus(self.bn2(self.fc2(x)) + self.bn2_b)
        x = F.softplus(self.fc3(x))
        return x

In [0]:
def get_triplets(prediction,size):
      a = prediction[0 : size] # query case (positive)
      b = prediction[size : 2 * size] # positive case
      c = prediction[2 * size : 3 * size] # negative

      return a,b,c

def loss_labeled(a,b,c):
        n_plus = torch.sqrt(torch.sum((a - b)**2, axis=1))
        n_minus = torch.sqrt(torch.sum((a - c)**2, axis=1))
        # print(n_plus.size(), n_minus.size())
        z = torch.cat((torch.unsqueeze(n_minus, 1), torch.unsqueeze(n_plus, 1)), 1)
        z = log_sum_exp(z, axis=1)
        loss_lab = -torch.mean(n_minus) + torch.mean(z)
        return loss_lab


In [0]:

class ImprovedGAN(object):
    def __init__(self, G, D, labeled, unlabeled, test, args):
        # if os.path.exists(args.savedir):
        #     print('Loading model from ' + args.savedir)
        #     self.G = torch.load(os.path.join(args.savedir, 'G.pkl'))
        #     self.D = torch.load(os.path.join(args.savedir, 'D.pkl'))
        # else:
        #     os.makedirs(args.savedir)
        #     self.G = G
        #     self.D = D
        #     torch.save(self.G, os.path.join(args.savedir, 'G.pkl'))
        #     torch.save(self.D, os.path.join(args.savedir, 'D.pkl'))
        # self.writer = tensorboardX.SummaryWriter(log_dir=args.logdir)
        self.G = G
        self.D = D
        if args.cuda:
            self.G.cuda()
            self.D.cuda()
        self.labeled = labeled
        self.unlabeled = unlabeled
        self.test = test
        self.Doptim = optim.Adam(self.D.parameters(), lr=args.lr, betas= (args.momentum, 0.999))
        self.Goptim = optim.Adam(self.G.parameters(), lr=args.lr, betas = (args.momentum,0.999))
        self.args = args
    

    def trainD(self, x_label, y, x_unlabel):
        
        x_label = Variable(x_label).cuda()
        x_unlabel = Variable(x_unlabel).cuda()
        y = Variable(y).cuda()
        #  y = Variable(y, requires_grad = False)
        
        output_label = self.D(x_label)
        output_unlabel = self.D(x_unlabel) 
        output_fake_G = self.G(x_unlabel.size()[0]).view(x_unlabel.size())
        output_fake = self.D(output_fake_G)  
        
        

        a_lab, b_lab, c_lab = get_triplets(output_label, 100)
        loss_triplet = loss_labeled(a_lab, b_lab, c_lab)
           
        loss_unsupervised = -0.5 * torch.mean(log_sum_exp(output_unlabel)) + 0.5 * torch.mean(F.softplus(log_sum_exp(output_unlabel))) + 0.5 * torch.mean(F.softplus(log_sum_exp(output_fake)))

        loss = loss_unsupervised + loss_triplet
        
        self.Doptim.zero_grad()
        loss.backward()
        self.Doptim.step()
        return loss.detach().cpu().numpy(), loss_unsupervised.detach().cpu().numpy(), loss_triplet.detach().cpu().numpy()
    
    def trainG(self, x_unlabel):
        
        fake = self.G(x_unlabel.size()[0]).view(x_unlabel.size())
        mom_gen, output_fake = self.D(fake, feature=True)
        mom_unlabel, _ = self.D(Variable(x_unlabel), feature=True)
        mom_gen = torch.mean(mom_gen, dim = 0)
        mom_unlabel = torch.mean(mom_unlabel, dim = 0)
        
        loss_fm = torch.mean(torch.square(torch.mean(mom_gen, axis = 0))) - torch.mean(mom_unlabel, axis = 0)
        loss = loss_fm 
        self.Goptim.zero_grad()
        self.Doptim.zero_grad()
        loss.backward()
        self.Goptim.step()
        return loss.detach().cpu().numpy()

    def train(self):
      if not os.path.exists('checkpoints_16_200_final'):
        os.makedirs('checkpoints_16_200_final')
     
      if args.load_saved == True:
        self.G.load_state_dict(torch.load(os.path.join('checkpoints_16_200_final', args.generator_save)))
        self.D.load_state_dict(torch.load(os.path.join('checkpoints_16_200_final', args.discriminator_save)))
      loss_g_list = []
      loss_triplet_list = []
      loss_disc_list = []
      loss_unsuper_list = []
      self.G.train()
      self.D.train()
      
      for epoch in range(self.args.epochs):
          unlabel_loader1 = DataLoader(self.unlabeled, batch_size = self.args.batch_size, shuffle = True, drop_last = True, num_workers = 4)
          unlabel_loader2 = DataLoader(self.unlabeled, batch_size = self.args.batch_size, shuffle = True, drop_last = True, num_workers = 4).__iter__()
          label_loader = DataLoader(MnistLabel(20, 60000), batch_size = 300, shuffle = False, drop_last=True, num_workers = 4).__iter__()
          loss_disc = 0
          loss_gen = 0
          batch_num = 0
          loss_triplet = 0
          loss_unsuper = 0
          
          for (unlabel1, label1) in unlabel_loader1:
              
              unlabel2, label2 = next(unlabel_loader2)
              x, y = next(label_loader)
              # if batch_num == 0:
              # print(batch_num, x.size(), y[0], y[34], y[98])
              x = x.cuda()
              y = y.cuda()
              unlabel1 = unlabel1.cuda()
              unlabel2 = unlabel2.cuda()

              # print(x.size(), y.size(), unlabel1.size())
              ld, lus, ltr = self.trainD(x, y, unlabel1)
              
              lg = self.trainG(unlabel2)
              if epoch > 1 and lg > 1:
                  lg = self.trainG(unlabel2)
              
              loss_disc += ld
              loss_gen += lg
              loss_unsuper += lus
              loss_triplet += ltr
              batch_num += 1
              
          
          loss_gen = loss_gen / batch_num
          loss_disc = loss_disc / batch_num
          loss_unsuper = loss_unsuper / batch_num
          loss_triplet = loss_triplet / batch_num 
          print(epoch, loss_gen, loss_disc, loss_unsuper, loss_triplet)
          loss_g_list.append(loss_gen)
          loss_triplet_list.append(loss_triplet)
          loss_unsuper_list.append(loss_unsuper)
          loss_disc_list.append(loss_disc)
          
          if (epoch + 1) % self.args.eval_interval == 0 and args.save == True:
              
              torch.save(self.G.state_dict(), os.path.join('checkpoints_16_200_final', args.generator_save))
              torch.save(self.D.state_dict(), os.path.join('checkpoints_16_200_final', args.discriminator_save))
          if epoch == 1:
              torch.save(self.G.state_dict(), os.path.join('checkpoints_16_200_final', 'Generator_fr_wrst_img'))
              torch.save(self.D.state_dict(), os.path.join('checkpoints_16_200_final', 'Disc_fr_wrst_img'))
          
          if epoch == 50:
              torch.save(self.G.state_dict(), os.path.join('checkpoints_16_200_final', 'Generator_fr_avg_img'))
              torch.save(self.D.state_dict(), os.path.join('checkpoints_16_200_final', 'Disc_fr_avg_img'))

      return loss_g_list, loss_triplet_list, loss_unsuper_list, loss_disc_list

    def predict(D):
      # D.load_state_dict(torch.load(os.path.join('checkpoints_16_200_exp', 'Discriminator')))
      D = self.D.cuda()
      train_loader = DataLoader(MnistUnlabel(), batch_size = 60000, shuffle = True, drop_last = True, num_workers = 4).__iter__()
      x, y = next(train_loader)
      x = x.cuda()
      y = y.cuda()
      feature_16_list = D(x)
      train_x = feature_16_list.detach().cpu().numpy()
      train_y = y.detach().cpu().numpy()
      test_loader = DataLoader(MnistTest(), batch_size = 10000, shuffle = True, drop_last = True, num_workers = 4).__iter__()
      x, y = next(test_loader)
      x = x.cuda()
      y = y.cuda()
      feature_16_list = D(x)
      test_x = feature_16_list.detach().cpu().numpy()
      test_y = y.detach().cpu().numpy()
      print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)
      
      neigh = KNeighborsClassifier(n_neighbors = 9)
      neigh.fit(train_x, train_y)
      pred_y = neigh.predict(test_x)
      print(accuracy_score(test_y, pred_y))
      # print(precision_score(test_y, pred_y, average = 'macro'))
      # Y = cdist(test_x,train_x)
      # ind = np.argsort(Y,axis=1)
      # prec = 0.0;
      # acc = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
      # # calculating statistics
      # for k in range(np.shape(test_y)[0]):
      #     class_values = train_y[ind[k,:]]
      #     y_true = (test_y[k] == class_values)
      #     y_scores = np.arange(y_true.shape[0],0,-1)
      #     ap = average_precision_score(y_true, y_scores)
      #     prec = prec + ap
      #     for n in range(len(acc)):
      #         a = class_values[0:(n+1)]
      #         counts = np.bincount(a)
      #         b = np.where(counts==np.max(counts))[0]
      #         if test_y[k] in b:
      #             acc[n] = acc[n] + (1.0/float(len(b)))
      # prec = prec/float(np.shape(test_x)[0])
      # acc= [x / float(np.shape(test_x)[0]) for x in acc]
      # print("Final results: ")
      # print("mAP value: %.4f "% prec)
      # for k in range(len(acc)):
      #     print("Accuracy for %d - NN: %.2f %%" % (k+1,100*acc[k]) )


In [0]:
class params():
  description = 'PyTorch Improved GAN'
  batch_size = 300           
  epochs = 100
  lr = 0.003
  momentum = 0.5
  cuda = False
  seed = 1
  log_interval = 100
  eval_interval = 1
  unlabel_weight = 1
  logdir = './logfile'
  savedir = './checkpoints' 
  load_saved = True
  save = True
  generator_save = 'Generator'
  discriminator_save = 'Discriminator'

args = params()
if torch.cuda.is_available():
  args.cuda = True
np.random.seed(args.seed)
gan = ImprovedGAN(Generator(100), Discriminator(), MnistLabel(10, 60000), MnistUnlabel(), MnistTest(), args)
loss_g, loss_triplet, loss_unsuper, loss_d = gan.train()

In [0]:
itr = []
for i in range(100):
  itr.append(i)
plt1, = plt.plot(itr, loss_g, label = 'Generator')
plt2, = plt.plot(itr, loss_d, label = 'Disriminator')
# plt3, = plt.plot(itr, loss_triplet, label = 'Triplet Loss')
plt.title("Loss ")
plt.legend(handles=[plt1, plt2])
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.show()

In [0]:
def predict(D):
      D.load_state_dict(torch.load(os.path.join('checkpoints_16_200_final', 'Discriminator')))
      D = D.cuda()
      train_loader = DataLoader(MnistUnlabel(), batch_size = 60000, shuffle = True, drop_last = True, num_workers = 4).__iter__()
      x, y = next(train_loader)
      x = x.cuda()
      y = y.cuda()
      feature_16_list = D(x)
      train_x = feature_16_list.detach().cpu().numpy()
      train_y = y.detach().cpu().numpy()
      test_loader = DataLoader(MnistTest(), batch_size = 10000, shuffle = True, drop_last = True, num_workers = 4).__iter__()
      x, y = next(test_loader)
      x = x.cuda()
      y = y.cuda()
      feature_16_list = D(x)
      test_x = feature_16_list.detach().cpu().numpy()
      test_y = y.detach().cpu().numpy()
      print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)
      
      neigh = KNeighborsClassifier(n_neighbors = 9)
      neigh.fit(train_x, train_y)
      pred_y = neigh.predict(test_x)
      print(accuracy_score(test_y, pred_y))
      print(precision_score(test_y, pred_y, average = 'macro'))
      Y = cdist(test_x,train_x)
      ind = np.argsort(Y,axis=1)
      prec = 0.0;
      acc = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
      # calculating statistics
      for k in range(np.shape(test_y)[0]):
          class_values = train_y[ind[k,:]]
          y_true = (test_y[k] == class_values)
          y_scores = np.arange(y_true.shape[0],0,-1)
          ap = average_precision_score(y_true, y_scores)
          prec = prec + ap
          for n in range(len(acc)):
              a = class_values[0:(n+1)]
              counts = np.bincount(a)
              b = np.where(counts==np.max(counts))[0]
              if test_y[k] in b:
                  acc[n] = acc[n] + (1.0/float(len(b)))
      prec = prec/float(np.shape(test_x)[0])
      acc= [x / float(np.shape(test_x)[0]) for x in acc]
      print("Final results: ")
      print("mAP value: %.4f "% prec)
      for k in range(len(acc)):
          print("Accuracy for %d - NN: %.2f %%" % (k+1,100*acc[k]) )


In [0]:
predict(D = Discriminator())

In [0]:
import matplotlib.pyplot as plt

def plot_images(G):
      
      G.load_state_dict(torch.load(os.path.join('checkpoints_16_200_final', 'Generator')))
      G = G.cuda()
      bst_image = G(2)[1]
      bst_image = bst_image.detach().cpu().numpy()
      bst_image = np.reshape(bst_image, (28, 28))
      plt.imshow(bst_image)
      plt.show()
           
      G.load_state_dict(torch.load(os.path.join('checkpoints_16_200_v2', 'Generator_fr_wrst_img')))
      G = G.cuda()
      wrst_image = G(2)[0]
      wrst_image = wrst_image.detach().cpu().numpy()
      wrst_image = np.reshape(wrst_image, (28, 28))
      print(wrst_image.shape)
      plt.imshow(wrst_image)
      plt.show()
      
      G.load_state_dict(torch.load(os.path.join('checkpoints_16_200_v2', 'Generator_fr_avg_img')))
      G = G.cuda()
      avg_image = G(2)[1]
      avg_image = avg_image.detach().cpu().numpy()
      avg_image = np.reshape(avg_image, (28, 28))
      plt.imshow(avg_image)
      plt.show()

In [0]:
plot_images(G = Generator(100))